In [1]:
#%load_ext autoreload
#%autoreload 2

#%load_ext dotenv
#%dotenv

In [2]:
import os
import pandas as pd
import sys
sys.path.append("../")
from utils import db_access
ybdata = db_access.YbData()
import sqlalchemy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import utils


In [3]:
pd.__version__

'1.5.3'

In [4]:
companies_carbon = pd.read_sql(
    """
SELECT target_id, fiscal_year, company_name, trbc_industrygroup,revenue,
co2_scope1_intensity,
co2_scope2_intensity,
co2_scope3_intensity,
(co2_scope1_intensity +
co2_scope2_intensity +co2_scope3_intensity) as total_intensity
FROM co2audit.companies_carbon_v14
WHERE fiscal_year >'2016'
GROUP BY target_id, fiscal_year, company_name, trbc_industrygroup,revenue,
co2_scope1_intensity,
co2_scope2_intensity,
co2_scope3_intensity
;

    
    
    """, ybdata.connection)


In [5]:
decarbonization_df=pd.DataFrame(companies_carbon, columns = ['target_id','fiscal_year', 'company_name','trbc_industrygroup', 'total_intensity', 'revenue'])
decarbonization_df.head()

,target_id,fiscal_year,company_name,trbc_industrygroup,total_intensity,revenue
0,0,2017,Apple Inc,"Computers, Phones & Household Electronics",122.566243,2.292340e+11
1,0,2018,Apple Inc,"Computers, Phones & Household Electronics",97.531730,2.655950e+11
2,0,2019,Apple Inc,"Computers, Phones & Household Electronics",99.532839,2.601740e+11
3,0,2020,Apple Inc,"Computers, Phones & Household Electronics",85.549492,2.745150e+11
4,0,2021,Apple Inc,"Computers, Phones & Household Electronics",66.117398,3.658170e+11


In [6]:
#industry_totals = decarbonization_df.groupby(by="trbc_industrygroup")["total_intensity"].sum()
#industry_totals.head()

decarbonization_df[decarbonization_df["fiscal_year"]>2016].head()


,target_id,fiscal_year,company_name,trbc_industrygroup,total_intensity,revenue
0,0,2017,Apple Inc,"Computers, Phones & Household Electronics",122.566243,2.292340e+11
1,0,2018,Apple Inc,"Computers, Phones & Household Electronics",97.531730,2.655950e+11
2,0,2019,Apple Inc,"Computers, Phones & Household Electronics",99.532839,2.601740e+11
3,0,2020,Apple Inc,"Computers, Phones & Household Electronics",85.549492,2.745150e+11
4,0,2021,Apple Inc,"Computers, Phones & Household Electronics",66.117398,3.658170e+11


In [7]:
sum_df = decarbonization_df.groupby(['fiscal_year','trbc_industrygroup']).agg({'revenue': 'sum'})
sum_df.head()

revenue
fiscal_year trbc_industrygroup                            
2017        Aerospace & Defense               7.121653e+11
            Automobiles & Auto Parts          3.703555e+12
            Banking Services                  5.462874e+12
            Beverages                         5.599583e+11
            Biotechnology & Medical Research  8.025754e+10

In [8]:
merged_df = pd.merge(
    decarbonization_df, sum_df.reset_index(),
    on=["trbc_industrygroup", "fiscal_year"], suffixes=(None, "_ig"))
merged_df.head()

,target_id,fiscal_year,company_name,trbc_industrygroup,total_intensity,revenue,revenue_ig
0,0,2017,Apple Inc,"Computers, Phones & Household Electronics",122.566243,2.292340e+11,1.411744e+12
1,151,2017,HP Inc,"Computers, Phones & Household Electronics",789.559321,5.205600e+10,1.411744e+12
2,207,2017,Western Digital Corp,"Computers, Phones & Household Electronics",61.989975,1.909300e+10,1.411744e+12
3,213,2017,Hewlett Packard Enterprise Co,"Computers, Phones & Household Electronics",435.639811,2.887100e+10,1.411744e+12
4,273,2017,NetApp Inc,"Computers, Phones & Household Electronics",30.115600,5.519000e+09,1.411744e+12


In [9]:
merged_df.loc[:, "weighted_intensity"] = merged_df.apply(
    lambda r: r["total_intensity"]*r["revenue"]/r["revenue_ig"], axis=1)
merged_df.head()

,target_id,fiscal_year,company_name,trbc_industrygroup,total_intensity,revenue,revenue_ig,weighted_intensity
0,0,2017,Apple Inc,"Computers, Phones & Household Electronics",122.566243,2.292340e+11,1.411744e+12,19.901880
1,151,2017,HP Inc,"Computers, Phones & Household Electronics",789.559321,5.205600e+10,1.411744e+12,29.113857
2,207,2017,Western Digital Corp,"Computers, Phones & Household Electronics",61.989975,1.909300e+10,1.411744e+12,0.838378
3,213,2017,Hewlett Packard Enterprise Co,"Computers, Phones & Household Electronics",435.639811,2.887100e+10,1.411744e+12,8.909095
4,273,2017,NetApp Inc,"Computers, Phones & Household Electronics",30.115600,5.519000e+09,1.411744e+12,0.117732


In [10]:
#merged_df.loc[merged_df["fiscal_year"]==2016].to_excel(
#"artifacts/company_carbon_year_merged_df.xlsx")

In [11]:
weighted_intensity = merged_df.groupby(
    ["trbc_industrygroup", "fiscal_year"]).apply(
    lambda g: g["weighted_intensity"].sum()).reset_index().rename(columns={0: "intensity"})

intensity_pct_change = weighted_intensity.set_index("fiscal_year").groupby(
    "trbc_industrygroup", group_keys=True).apply(
        lambda g: g[["intensity"]].pct_change())

weighted_intensity = weighted_intensity.set_index(["trbc_industrygroup", "fiscal_year"]).join(
    intensity_pct_change.rename(columns={"intensity": "pct_change"}))

In [12]:
weighted_intensity.head()

intensity  pct_change
trbc_industrygroup  fiscal_year                         
Aerospace & Defense 2017          125.283843         NaN
                    2018          148.977435    0.189119
                    2019         2705.713615   17.161902
                    2020         2035.973696   -0.247528
                    2021         2282.324504    0.120999

In [13]:
intensity_pct_change.head()

intensity
trbc_industrygroup  fiscal_year           
Aerospace & Defense 2017               NaN
                    2018          0.189119
                    2019         17.161902
                    2020         -0.247528
                    2021          0.120999

In [14]:
pd.set_option('display.max_rows', None)
weighted_intensity.head()

intensity  pct_change
trbc_industrygroup  fiscal_year                         
Aerospace & Defense 2017          125.283843         NaN
                    2018          148.977435    0.189119
                    2019         2705.713615   17.161902
                    2020         2035.973696   -0.247528
                    2021         2282.324504    0.120999

In [15]:
#weighted_intensity.to_excel("artifacts/trbc_industrygroup_intensities_year.xlsx")

In [16]:
weighted_intensity.reset_index(inplace=True)
min_intensity = weighted_intensity.loc[weighted_intensity.groupby(['trbc_industrygroup'], sort=False)['fiscal_year'].idxmin()][['trbc_industrygroup','intensity']].rename(columns={'intensity': "2017"})
min_intensity.head()

,trbc_industrygroup,2017
0,Aerospace & Defense,125.283843
5,Automobiles & Auto Parts,1158.324382
10,Banking Services,31.424545
15,Beverages,454.536886
20,Biotechnology & Medical Research,42.230729


In [17]:
max_intensity = weighted_intensity.loc[weighted_intensity.groupby(['trbc_industrygroup'], sort=False)['fiscal_year'].idxmax()][['trbc_industrygroup','intensity']].rename(columns={'intensity': "2020_or_2021"})
max_intensity.head()

,trbc_industrygroup,2020_or_2021
4,Aerospace & Defense,2282.324504
9,Automobiles & Auto Parts,1403.100001
14,Banking Services,55.404426
19,Beverages,506.329542
24,Biotechnology & Medical Research,62.398627


In [18]:
max_intensity.index = min_intensity.index
min_intensity['2020_or_2021']=max_intensity['2020_or_2021']
min_intensity.head()

,trbc_industrygroup,2017,2020_or_2021
0,Aerospace & Defense,125.283843,2282.324504
5,Automobiles & Auto Parts,1158.324382,1403.100001
10,Banking Services,31.424545,55.404426
15,Beverages,454.536886,506.329542
20,Biotechnology & Medical Research,42.230729,62.398627


In [19]:
min_intensity['change'] = (min_intensity['2020_or_2021']- min_intensity['2017'])/min_intensity['2017']
min_intensity.head()

,trbc_industrygroup,2017,2020_or_2021,change
0,Aerospace & Defense,125.283843,2282.324504,17.217229
5,Automobiles & Auto Parts,1158.324382,1403.100001,0.211319
10,Banking Services,31.424545,55.404426,0.763094
15,Beverages,454.536886,506.329542,0.113946
20,Biotechnology & Medical Research,42.230729,62.398627,0.477565


In [20]:
min_intensity.to_excel("intensity_change_yer.xlsx")

In [23]:
min_intensity['change'].describe()

count    60.000000
mean      2.831864
std      10.625138
min      -0.746245
25%       0.193057
50%       0.607805
75%       1.529079
max      80.751126
Name: change, dtype: float64